no sys args version, for debug purpose

Following block are complete process for running KMC on API=1, from generating input files to run the kernel, Always run it see if it works after make modification on code.


following block is for generating the local cluster expansion model. See the Table S3 in support information of nasicon kmc as reference

The localclusterexpansion.initialization is for generating the **lce.json and lcesite.json**

notice the dir has changed to ../examples

**after running the blocks, if no file is updated, meaning that the scripts is doint correct thing**
**because the file generated by the script is the same as the sample input, i.e., work as expected**

In [1]:
from kmcpy.model import LocalClusterExpansion
import os 
os.chdir("../examples")
a=LocalClusterExpansion()
a.initialization(center_Na1_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='../dev_test/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a.to_json("inputs/lce.json")

Searching local env around [ 0.      0.     11.0905] Na ...
793 clusters will be generated ...

Generating possible clusters within this diffusion unit...
Cutoffs: pair = 6 Angst, triplet = 6 Angst, quadruplet = 6 Angst
Type	Index	max_length	min_length	Point Group	Multiplicity
point	(0,)      	   0.000	   0.000	Kh   	    6
point	(6,)      	   0.000	   0.000	Kh   	    6
pair 	(0, 1)    	   4.727	   4.727	D*h  	    6
pair 	(0, 3)    	   5.103	   5.103	D*h  	    6
pair 	(0, 6)    	   3.710	   3.710	C*v  	    6
pair 	(0, 7)    	   3.056	   3.056	C*v  	    6
pair 	(0, 8)    	   3.162	   3.162	C*v  	    6
pair 	(6, 7)    	   4.922	   4.922	D*h  	    6
pair 	(6, 9)    	   5.629	   5.629	D*h  	    6
triplet	(0, 1, 2) 	   5.103	   4.727	C2v  	    6
triplet	(0, 1, 6) 	   4.727	   3.162	Cs   	    6
triplet	(0, 1, 7) 	   4.727	   3.056	Cs   	    6
triplet	(0, 3, 4) 	   5.103	   5.103	D3h  	    2
triplet	(0, 3, 7) 	   5.103	   3.056	C2v  	    6
triplet	(0, 6, 7) 	   4.922	   3.056	Cs   	    6
tripl

the kmc.event_generator is for generating the **event_kernal.csv**

read the primitive cell from prim.json, and write events.json as output

In [4]:
from kmcpy.event_generator import generate_events

generate_events(prim_fname=r"../examples/inputs/prim.json",supercell_shape=[2,1,1],event_fname="./events.json",event_kernal_fname='./event_kernal.csv')

Generating events ...
Initializing model with pirm.json at ../examples/inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting supercell ...
Looking for all possible swaps with ncpus = 40  (this might take a while) ...
Saving: ./events.json
Generating event kernal ...
Saving into: ./event_kernal.csv


In [3]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=1,**kwargs):
    inputset=InputSet.from_json("./test_input.json")

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    if api==1:


        inputset.set_parameter("occ",load_occ(inputset._parameters["mc_results"],inputset._parameters["supercell_shape"],api=inputset.api))
        # step 1 initialize global occupation and conditions
        kmc = KMC()
        events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

        # # step 2 compute the site kernal (used for kmc run)
        kmc.load_site_event_list(inputset._parameters["event_kernel"])

        # # step 3 run kmc
        kmc.run_from_database(events=events_initialized,**inputset._parameters)
    if api>1:
        raise NotImplementedError
    pass


if __name__ == "__main__":

    main()


FileNotFoundError: [Errno 2] No such file or directory: './test_input.json'

Api version 2 initiated
aiming for more flexible inputs.
Following block is the main KMC function
DO make sure that they provide similar result as in API=1

In [ ]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=2,**kwargs):
    inputset=InputSet.from_json("test_input_v2.json",api=2)

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    inputset.set_parameter("occ",load_occ(fanme=inputset._parameters["mc_results"],shape=inputset._parameters["supercell_shape"],select_sites=inputset._parameters["select_sites"],api=inputset.api,verbose=True))
    
    
    

    # step 1 initialize global occupation and conditions
    
    
    
    kmc = KMC(api=api)
    events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

    # # step 2 compute the site kernal (used for kmc run)
    kmc.load_site_event_list(inputset._parameters["event_kernel"])

    # # step 3 run kmc
    kmc.run_from_database(events=events_initialized,**inputset._parameters)


if __name__ == "__main__":

    main()